Install dependencies

In [ ]:
!pip install pandas
!pip install numpy
!pip install jupyter-dash
!pip install matplotlib
!pip install plotly

Import requirements

In [64]:
import os
import glob
import pandas as pd
import plotly.express as px

Define event dates

In [46]:
class Event:
    def __init__(self, date, annotation, line_type, color):
        self.date = date
        self.annotation = annotation
        self.line_type = line_type
        self.color = color

events = [
    Event("2017-12-24", "Christmas 2017", "dash", "red"),
    Event("2018-12-24", "Christmas 2018", "dash", "red"),
    Event("2019-12-24", "Christmas 2019", "dash", "red"),
    Event("2017-12-31", "New Year 2017", "dash", "blue"),
    Event("2018-12-31", "New Year 2018", "dash", "blue"),
    Event("2019-12-31", "New Year 2019", "dash", "blue")
]

Load chart data to chart dictionary

In [76]:
# Read all files in path
charts = {}

path = r'data'
all_files = glob.glob(path + "/chart_*.csv")
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    if df.shape[0] > 0:
        charts[os.path.basename(filename).rsplit(".")[0]] = df

# Specific chart data can be accessed like:
# chartDataDe = charts["chart_de"]
# chartDataGlobal = charts["chart_global"]

Load and join track feature data

In [73]:
featureData = pd.read_csv('data/track_features.csv')

In [74]:
def join_features(chart: pd.DataFrame, feature: pd.DataFrame):
    return chart.merge(feature, left_on='track_id', right_on='track_id')

In [78]:
feature_charts = {}
for key in charts:
   feature_charts[key] =  join_features(charts[key], featureData)

# Specific chart feature data can be accessed like:
# feature_charts_de = feature_charts["chart_de"]
# feature_charts_global = feature_charts["chart_global"]

In [79]:
ndmean = feature_charts["chart_de"].groupby('date').mean()
ndmeanGlobal = feature_charts["chart_global"].groupby('date').mean()
ndmeanIt = feature_charts["chart_it"].groupby('date').mean()

In [92]:
def collate_sets(chart_set, region, feature):
    collate_df = pd.DataFrame()
    if region.lower() == "europe".lower():
        collate_df.insert(0, "Germany", chart_set["chart_de"][feature])
        collate_df.insert(0, "Italy", chart_set["chart_it"][feature])
        collate_df.insert(0, "Finland", chart_set["chart_fi"][feature])
        collate_df.insert(0, "Belgium", chart_set["chart_be"][feature])
    return collate_df

Visualization

In [93]:
mean_feature_charts = {}
for key in feature_charts:
   mean_feature_charts[key] = feature_charts[key].groupby('date').mean()

tempDf = collate_sets(mean_feature_charts, "europe","duration_ms")

fig = px.line(tempDf)
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Duration (ms)",
    legend_title="Country"
)

# for event in events:
#     fig.add_vline(x=event.date,line_width=1, line_dash=event.line_type, col='all', line_color=event.color)

fig.show()